In [2]:
%pip install langgraph langchain openai


  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached charset_normalizer-3.4.2-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.7.9-py3-none-any.whl.metadata (2.4 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 26.9 MB/s eta 0:00:00
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp312-cp312-win_amd64.whl (105 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta

In [ ]:
import os
import openai
import sys
# sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['API_KEY']

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

In [ ]:
# DeepSeek-r1 model
from openai import OpenAI

client = OpenAI(api_key=os.environ['API_KEY'], base_url=os.environ['BASE_URL'])

response = client.chat.completions.create(
    model="r1w8a8",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)


Okay, the user just said "Hello". I should respond in a friendly and welcoming manner. Maybe ask how I can assist them today. Keep it open-ended so they feel comfortable to ask anything. Let me make sure the response is warm and approachable.

Hmm, should I use an exclamation mark to sound more enthusiastic? Yeah, "Hello! How can I assist you today?" sounds good. It's simple and to the point. I don't want to overwhelm them with too much text. Just let them know I'm here to help. Yep, that works.
</think>

Hello! How can I assist you today?


In [ ]:
#在工单接收节点添加了信息不足反馈机制，但并不包括复杂逻辑以及RAG
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI  # 可替换为DeepSeek-R1
import json

# ----------- 状态定义 -----------
class State(TypedDict):
    user_input: str
    missing_fields: Optional[list]
    json_output: Optional[dict]

# ----------- 初始化模型 -----------
llm = ChatOpenAI(model="gpt-4", temperature=0)  # 替换为 DeepSeek-R1 接口也可

# ----------- 节点定义 -----------

# 1. 检查输入完整性
def check_completeness(state: State) -> State:
    prompt = f"""
你是一位城市治理智能助理，现在收到一条来自环卫工人的描述，请你判断该描述是否包含以下三类信息：
1. 时间信息
2. 地点信息
3. 遗洒情况

请以如下 JSON 结构输出判断：
{{
  "time": true/false,
  "location": true/false,
  "incident": true/false,
  "missing_fields": ["time", "location"]  // 若有缺失，请列出
}}

描述如下：
{state['user_input']}
    """

    response = llm.invoke([HumanMessage(content=prompt)])
    result = json.loads(response.content)
    state["missing_fields"] = result.get("missing_fields", [])
    return state

# 2. 提示用户补充
def ask_for_more(state: State) -> State:
    missing = ", ".join(state["missing_fields"])
    prompt = f"""
您好，您提供的描述缺少以下信息：{missing}。
请您补充这部分内容，例如：
- 时间：今天是什么时候发现的？
- 地点：具体是在哪条路或哪个路段？
- 情况：大概有多严重？是否影响交通？

请补充完整描述：
    """
    print(prompt)  # 或发回前端交互界面
    new_input = input("请输入补充内容：")  # 模拟人类用户回复
    state["user_input"] += " " + new_input
    return state

# 3. 生成 JSON 表单
def generate_json(state: State) -> State:
    prompt = f"""
你是一位城市治理专家，请从以下描述中提取关键信息，并输出如下 JSON 格式：
{{
  "time": "...",
  "location": "...",
  "description": "...",
  "reporter": "...",
  "images": []
}}
描述如下：
{state['user_input']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    state["json_output"] = json.loads(response.content)
    print("✅ 表单生成成功：", state["json_output"])
    return state

# ----------- LangGraph 状态图 -----------
workflow = StateGraph(State)

workflow.add_node("check_completeness", check_completeness)
workflow.add_node("ask_for_more", ask_for_more)
workflow.add_node("generate_json", generate_json)

workflow.set_entry_point("check_completeness")

# 动态路由判断是否完整，判定条件
def is_complete(state: State) -> str:
    if not state["missing_fields"]:
        return "generate_json"
    else:
        return "ask_for_more"

workflow.add_conditional_edges("check_completeness", is_complete)
workflow.add_edge("ask_for_more", "check_completeness")  # 多轮补充
workflow.add_edge("generate_json", END)

app = workflow.compile()

# ----------- 执行示例 -----------
initial_state = {
    "user_input": "我在科技园看到路上很多泥，可能是泥头车干的",
    "missing_fields": None,
    "json_output": None,
}

result = app.invoke(initial_state)


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI  # 替换为 DeepSeek-R1 的兼容模型
import json
import datetime

# 1. 定义状态结构
class EventState(TypedDict):
    raw_input: str
    filled_input: str
    json_data: Optional[dict]
    missing_fields: list
    assigned_entity: Optional[str]
    dispatch_status: Optional[str]

# 2. 初始化模型（可替换为 DeepSeek-R1）
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 3. 节点定义

def receive_report(state: EventState) -> EventState:
    state['filled_input'] = state['raw_input']
    return state

def check_and_complete_info(state: EventState) -> EventState:
    prompt = f"""
你是一位城市治理智能助理，现在收到一条描述，请判断是否包含以下信息：时间、地点、事件情况。
请以如下格式返回：
{{
  "time": true/false,
  "location": true/false,
  "incident": true/false,
  "missing_fields": ["time", "location"]
}}
描述如下：
{state['filled_input']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    result = json.loads(response.content)
    state['missing_fields'] = result.get("missing_fields", [])
    return state

def ask_for_more(state: EventState) -> EventState:
    missing = ", ".join(state['missing_fields'])
    print(f"⚠️ 信息缺失：{missing}，请补充如下内容。")
    user_reply = input("请输入补充描述：")
    state['filled_input'] += " " + user_reply
    return state

def extract_json(state: EventState) -> EventState:
    prompt = f"""
请从下列事件描述中提取如下 JSON 信息：
{{
  "time": "",
  "location": "",
  "description": "",
  "reporter": "",
  "images": []
}}
描述如下：
{state['filled_input']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    state['json_data'] = json.loads(response.content)
    return state

def assign_responsibility(state: EventState) -> EventState:
    prompt = f"""
你是一位城市治理责任判定专家，请判断事件责任应归属于哪一类：运输企业、工地企业或环卫公司。
返回格式：{{"responsibility": "运输企业", "reason": "..."}}
描述如下：
{state['filled_input']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    result = json.loads(response.content)
    state['assigned_entity'] = result['responsibility']
    return state

def dispatch_task(state: EventState) -> EventState:
    now = datetime.datetime.now()
    deadline = now + datetime.timedelta(hours=2)
    dispatch_record = {
        "event_id": now.strftime("%Y%m%d%H%M%S"),
        "time": state['json_data'].get("time", str(now)),
        "location": state['json_data'].get("location", "未知"),
        "description": state['json_data'].get("description", ""),
        "dispatched_to": state['assigned_entity'],
        "dispatch_deadline": deadline.strftime("%Y-%m-%d %H:%M"),
        "status": "已派遣"
    }
    state['dispatch_status'] = json.dumps(dispatch_record, ensure_ascii=False, indent=2)
    print("✅ 派遣成功，记录如下：")
    print(state['dispatch_status'])
    return state

# 4. 构建LangGraph
workflow = StateGraph(EventState)

workflow.add_node("receive_report", receive_report)
workflow.add_node("check_and_complete_info", check_and_complete_info)
workflow.add_node("ask_for_more", ask_for_more)
workflow.add_node("extract_json", extract_json)
workflow.add_node("assign_responsibility", assign_responsibility)
workflow.add_node("dispatch_task", dispatch_task)

workflow.set_entry_point("receive_report")

def is_info_complete(state: EventState) -> str:
    return "extract_json" if not state['missing_fields'] else "ask_for_more"

workflow.add_conditional_edges("check_and_complete_info", is_info_complete)
workflow.add_edge("receive_report", "check_and_complete_info")
workflow.add_edge("ask_for_more", "check_and_complete_info")
workflow.add_edge("extract_json", "assign_responsibility")
workflow.add_edge("assign_responsibility", "dispatch_task")
workflow.add_edge("dispatch_task", END)

app = workflow.compile()

# 5. 示例运行
if __name__ == "__main__":
    user_input = input("请输入泥头车遗撒事件描述：")
    initial_state = {
        "raw_input": user_input,
        "filled_input": "",
        "json_data": {},
        "missing_fields": [],
        "assigned_entity": None,
        "dispatch_status": None
    }
    app.invoke(initial_state)
